# Topic Modeling

Adapted from Heidi Smith's File (Topic_Model_Attempt_1.ipynb) and from the 2021 Data+ team's code (https://github.com/ABeeShake/Ethical-Consumption-Before-Capitalism/blob/main/topic%20modelling/code_actualrunning_awsvm.py)

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

In [55]:
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import pandas as pd
import gensim.corpora as corpora
from pprint import pprint

from csv import writer
import re
import os

In [58]:
directory = '/home/rapiduser/Materials/Texts/'
file = directory + 'post-restoration.csv'
topicModel = directory + '/topic model'
results = '/home/rapiduser/Materials/results'

In [59]:
# Read in csv
df = pd.read_csv(file)

In [60]:
# Remove stopwords/preprocess
stop_words = stopwords.words('english')
stop_words.extend(['thus', 'thereof', 'thence', 'thee', 'therein', 
                    'wherein', 'whereby', 'whereas', 'also', 'us', 'upon', 
                    'would', 'within', 'indeed', 'become', 'viz', 'per', 'anno', 
                    'whilst', 'thoe', 'ome', 'uch', 'said', 'shall', 'hath',
                    'may','made','much','one'])

def preprocess(data):
    for text in data:
        yield(gensim.utils.simple_preprocess(str(text)))
        
def remove_stopwords(data):
    return [[word for word in simple_preprocess(str(doc))
            if word not in stop_words] for doc in data]

In [61]:
data = df.text.values.tolist()
data = list(preprocess(data))
data = remove_stopwords(data)

# Creation of corpus
# Dictionary
id2word = corpora.Dictionary(data)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data]
# Train LDA model
num_topics = 10

# Build model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=num_topics)


In [72]:
import pyLDAvis
import pyLDAvis.gensim_models
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, results+'/post-restoration.html')
LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.009063 -0.002535       1        1  20.706928
8      0.004458 -0.005791       2        1  16.133056
1      0.006611  0.000490       3        1  14.802736
0     -0.003424  0.000473       4        1  10.953459
5      0.004610  0.001103       5        1  10.579215
3     -0.000944  0.003754       6        1  10.492616
7     -0.000480 -0.003315       7        1   7.370328
2      0.001000  0.002104       8        1   3.010181
4     -0.002064  0.001035       9        1   2.976530
6     -0.000704  0.002682      10        1   2.974951, topic_info=            Term         Freq        Total Category  logprob  loglift
47           aid  4061.000000  4061.000000  Default  30.0000  30.0000
680         king  3744.000000  3744.000000  Default  29.0000  29.0000
553        great  3634.000000  3634.000000  Default  28.0000  28.0000
1153        time  2881.000000  2881.000000  Default  27.0000  27.0000
22011  plaintiff  2135.000000  2135.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
269      country    44.145323  1690.457079  Topic10  -6.0942  -0.1303
5592          mr    40.986302  1470.258872  Topic10  -6.1684  -0.0650
5195        lord    38.555605  1301.279332  Topic10  -6.2296  -0.0041
1379       house    38.891374  1937.880734  Topic10  -6.2209  -0.3936
1160       trade    38.694906  1855.977874  Topic10  -6.2260  -0.3555

[916 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
17522      1  0.355391    aaulted
17522      6  0.355391    aaulted
29681      1  0.287306    abeeles
29681      2  0.191538    abeeles
29681      3  0.095769    abeeles
...      ...       ...        ...
14763     10  0.048697    zenodor
33892      1  0.213349  zuinglius
33892      2  0.213349  zuinglius
33892      3  0.213349  zuinglius
33892      7  0.213349  zuinglius

[4208 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 2, 1, 6, 4, 8, 3, 5, 7])

In [ ]:
medical = re.compile('corruption|consumption|vein|circulation|body|corrupt|consume|wasting|waste|blood|physician|decay|canker|cancer|disease|illness|remedy|cure|sickness|hepatitis|fever|spirit|brain|mind|vital|therapeutic|degeneration|degenerate|plague|smells|putrid|bad|tetrid|breathing|wholesome|healthy|unhealthy|sane|insane|nervous|languish|faculties|enfeeblement|drinks|tuberculosis|constitution|bile|black bile|yellow bile|phlegm|lung|sores|fog|smoke| diagnosis|prognosis |fiber |atrophy |morbid|mortal| mortality|nerves|inanition|defect|distemper|swelling|upset| stomach|cough|exercise| unwholesome|evacuation|fatal|fatality|vessels|hemorrhage|bleeding|bleed| melancholy|diabetes|asthma|vomit|opiate|opium|ulcer|envy|jealousy|spoil|liver|vein|supple|heart|mouth|cured|pox')
political = re.compile('commonwealth|politic|political|body politic|government|prince|war|ruler|king|queen|extortion|oppression|restoration|revolution|constitution|navy|military|army|strength|power|might|england|britain|ireland|netherlands|scotland|netherlands|dutch|spanish|germany|holy roman empire|mughal|christian|christianity|god|state|country|parliament|crisis|house of commons|house of lords|lord|crisis|national|governor|public|publicke|low countries|controversy|controversies|authority|empire|bills|treatise|treatises|civilian|protection|pirates|pirate|civil war|massacre|indonesia|india|china|argier|tunis|kingdom|provincial|scandalous|scandal|majesty|subject|christiandom|enemy|turkish|ally|public|patent|trial|law|case|counsel|court|just|unjust|equal|good|evil|justice|natural|rights|virtue|jury|juries|jurisprudence|ethics|abuse|unethical|unlawful|statute|vicious|vice|cruel|prison|extortion|oppression|slave|race|nation|cruelty|grief|punishment|penalty|retribution')

economic = re.compile("economy|economic|money|monies|bullion|bullionist|mercantile|fiscal|taxation|exchange|corporation|corporate|company|merchant|joint-stock|merchants|currency|consumption|consume|consuming|consumables|trade|traffic|traffique|commerce|commercial|price|prices|spending|east indies|east india company|east-india company|eat-india company|east-india-company|monopoly|monopolization|monopolies|monopolium|monopolion|monopolie|monopolist|monopolizer|monopolize|monopolizes|monopolye|monopolyes|debt|credit|debtor|creditor|decoctor|bank|banks|usury|interest rate|interest|interest rates|importation|exportation|coin|employment|austerity|goods|treasure|commodities|income|commodity|revenue|land|profitable|unprofitable|industry|work|usurer|estate|property|substance|consumer|free trade")
religious = re.compile('Christian|charity|luxury|avarice|greed|vanity|pride|sloth|gluttony|simony|fraud|incest|theft|deceit|lucrum|cessans|stations|station|silk|velvet|damask|cinnamon|pepper|ginger|sumptuary|laws|abnegatio|self-denial|mendicancy|sin|root|evil|render|caesar|heresy|prodigality|prodigal||seed|soil|crop|yield|kingdom|field|sow|sowed|wheat|barn|weeds|enemy|treasure|treasures|rust|steel|heart|gold|silver|copper|belt|belts|worker|staff|journey|thorns|deceit|wealth|rich|riches|green|righteous|righteousness|thrive|leaf|leaves|wise|store|gulp|choice|food|oil|glance|sprout|sky|eagle|endure|crown|generation|secure|trust|security|fortune|sun|radiance|moon|splendor|rich|arrogant|hope in wealth|tower|pleasant|palace|deceive|tempt|free|money|content|material possession|in need|goods|brother|sister|eve|adam|samson|delilah|david|bathsheba|lust|lustful|lustfully|sin|devil|lechery|whore|cuckold|unnatural|natural|obey|disobey|deceive|cheat|cheating|modesty|sermon|christ|heaven|godly|saint|altar|candles|canon|mass|sacrament|cross|pastor|ritual|cleansing|church|salvation|savior|jesus|lord|witness|saved|communion|wine|bread|faith|anathema|anointing|apostle|apocalypse|atonement|baptism|holy|sacred|spirit|bishop|born-again|calvinist|evangelical|protestant|catholic|covenant|conviction|creed|demon|deacon|disciple|disciples|satan|satanic|fellowship|gospel|hallelujah|hell|indulgence|indulgent|justify|justification|lucifer|messiah|manifestation|god|zion|ordained|ordinance|ordinate|congregation|parish|parishioner|prophet|repent|sanctuary|redeem|redeemed|sanctified|sanctify|second|coming|testament|tribulation|trinity|words|bible|word|grace|absolution|adultery|obedience|anoint|antichrist|archangel|armageddon|ascension|atone|vision|biblical|blasphemy|bless|blessing|blessed|chalice|chapel|chaplain|cherub|condemnation|condemn|confession|confess|conscience|consecration|contrite|contrition|damnation|damned|damn|day|divine|doctrine|ecumenical|epistle|eternal|evangelicalism|excommunication|exile|resurrection|forgive|forgiveness|freedom|fundamental|gentile|revelation|heresy|Jehovah|judgment|judaism|supper|liturgy|ministry|missionary|mission|ordination|orthodox|pagan|pagans|paganism|passover|papacy|pope|christianity|abraham|penance|genesis|exodus|leviticus|numbers|deuteronomy|moses|union|pray|prayer|predestination|prophecy|psalm|psalms|providence|purgatory|rapture|reconciliation|reconcile|redemption|reform|reformed|reincarnation|reincarnate|resurrect|roman|rome|sabbath|sacrifice|sacrifices|satanism|save|saved|sinful|nature|creation|create|death|offering|offerings|tongues|soul|commandments|transgression|universal|moral|venial|virgin|perfect|vulgate|worship|scripture|scriptures|priesthood|ten|twelve|annihilate|proverb|proverbs|matthew|mark|luke|john|peter|john|james|samuel|timothy|isaiah|hebrew|hebrews|job|slave|slaves|human|reap|holiness|parable|parables|knowledge|seed|seeds|samaritan|sown|abundance|persecution|deceitful|fruitful|unfruitful|plant|planted|reject|rejected|integrity|rejoice|rejoiced|wealth|wicked|destruction|grievance|desire|desires|entice|enticed|unfaithful|tenant|tenants|guilt|guilty|splendor|curse|almighty|defile|defiled|falsehood|false|lie|lies|confront|confronts|contempt')